### Mediapipe Documentation: https://ai.google.dev/edge/mediapipe/solutions/guide ### 

## 0. Setup and Imports ##

In [5]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
# IMPORTS
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


# 1. Make Detections #

In [7]:
# VIDEO FEED
cap = cv2.VideoCapture(1) # BEWARE might have to change number to 0, 1, or -1

# Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    if not cap.isOpened():
        print("Error: Could not open video stream.")
    else:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            # Detect stuff and render
            ## Recolor image
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                print(landmarks)
            except:
                pass # maybe change further into project as we want to assess form accuracy

            # Reder detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )

            # print(results) to verify wireframe

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'): # Press 'Q' to exit
                break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1730032270.078416 11073052 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2
W0000 00:00:1730032270.238921 11171764 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730032270.262798 11171766 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[x: 0.5655478239059448
y: 0.6112264394760132
z: -0.9064927697181702
visibility: 0.9991705417633057
, x: 0.580129086971283
y: 0.5647822022438049
z: -0.8660745024681091
visibility: 0.9980663657188416
, x: 0.5896874666213989
y: 0.5652096271514893
z: -0.8660343885421753
visibility: 0.9979233741760254
, x: 0.5977968573570251
y: 0.5661970376968384
z: -0.8660731911659241
visibility: 0.9962320923805237
, x: 0.5494487285614014
y: 0.5652231574058533
z: -0.8934528827667236
visibility: 0.9987459182739258
, x: 0.5384209156036377
y: 0.5653268098831177
z: -0.8936392664909363
visibility: 0.9989468455314636
, x: 0.5260717272758484
y: 0.5657644867897034
z: -0.8939247131347656
visibility: 0.9986085295677185
, x: 0.5997639894485474
y: 0.5968379974365234
z: -0.602184534072876
visibility: 0.9964922070503235
, x: 0.48638027906417847
y: 0.6007830500602722
z: -0.7228402495384216
visibility: 0.9994580149650574
, x: 0.5772681832313538
y: 0.6707326769828796
z: -0.8032316565513611
visibility: 0.9986693859100342
, 

## 2. Determining Joints ##

In [ ]:
 ##JOINT COORDINATE NOTES
'''
    0. nose                         17. left_pinky
    1. left_eye_inner               18. right_pinky
    2. left_eye                     19. left_index
    3. left_eye_outer               20. right_index
    4. right_eye_inner              21. left_thumb
    5. right_eye                    22. right_thumb
    6. right_eye_outer              23. left_hip
    7. left_ear                     24. right_hip
    8. right_ear                    25. left_knee
    9. mouth_left                   26. right_knee
    10. mouth_right                 27. left_ankle
    11. left_shoulder               28. right_ankle
    12. right_shoulder              29. left_heel
    13. left_elbow                  30. right_heel
    14. right_elbow                 31. left_foot_inner
    15. left_wrist                  32. right_foot_inner
    16. right_wrist
'''


In [8]:
len(landmarks) # should get 33 since there are 33 coordinates

33

In [13]:
# CHANGE FROM VIDEO, had to add .value and .name to get it printing right 
for lndmrk in mp_pose.PoseLandmark:
    print(f"{lndmrk.value}: {lndmrk.name}")

0 : NOSE
1 : LEFT_EYE_INNER
2 : LEFT_EYE
3 : LEFT_EYE_OUTER
4 : RIGHT_EYE_INNER
5 : RIGHT_EYE
6 : RIGHT_EYE_OUTER
7 : LEFT_EAR
8 : RIGHT_EAR
9 : MOUTH_LEFT
10 : MOUTH_RIGHT
11 : LEFT_SHOULDER
12 : RIGHT_SHOULDER
13 : LEFT_ELBOW
14 : RIGHT_ELBOW
15 : LEFT_WRIST
16 : RIGHT_WRIST
17 : LEFT_PINKY
18 : RIGHT_PINKY
19 : LEFT_INDEX
20 : RIGHT_INDEX
21 : LEFT_THUMB
22 : RIGHT_THUMB
23 : LEFT_HIP
24 : RIGHT_HIP
25 : LEFT_KNEE
26 : RIGHT_KNEE
27 : LEFT_ANKLE
28 : RIGHT_ANKLE
29 : LEFT_HEEL
30 : RIGHT_HEEL
31 : LEFT_FOOT_INDEX
32 : RIGHT_FOOT_INDEX


### 2a. In this case, we are collection joints for BICEP based on video. This is to test joint calculations, we can always append more workouts ###

In [14]:
# coordinates of left shoulder
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7222025394439697
y: 0.8650181889533997
z: -0.21602007746696472
visibility: 0.9982003569602966

In [18]:
# coordinate value of left shoulder. You can do with all joints 
mp_pose.PoseLandmark.LEFT_SHOULDER.value

11

In [19]:
# coordinate  of left elbow
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.8240774273872375
y: 1.2897844314575195
z: -0.20187155902385712
visibility: 0.13615025579929352

In [20]:
# coordinate value of left wrist
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.8337461948394775
y: 1.6272392272949219
z: -0.533139169216156
visibility: 0.08433883637189865

# 3. Calculate Angles # 

In [26]:
def calculate_angle(a, b, c):
    a = np.array(a) # First point, shoulder
    b = np.array(b) # Mid point, elbow
    c = np.array(c) # Last point, wrist

    # Subtracting y values (wrist), converting to angle 
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians*100.0/np.pi)

    # Max angle for bicep curl should be 180 degrees. Converting if so 
    if angle > 100.0:
        angle = 360-angle
    
    return angle

In [28]:
# Grabbing only x value and y value of joints by using .x and .y
# can also use .z and .visibility 
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

In [29]:
shoulder, elbow, wrist

([0.6528438329696655, 1.7161941528320312],
 [0.7222025394439697, 0.8650181889533997],
 [0.8240774273872375, 1.2897844314575195])

In [31]:
calculate_angle(shoulder, elbow, wrist)

10.080793333202898

In [33]:
tuple(np.multiply(elbow, [640, 480]).astype(int)) # normalized coordinates for webcame 

(462, 415)

In [37]:
# UPDATING code from above to make detections

# VIDEO FEED
cap = cv2.VideoCapture(1) # BEWARE might have to change number to 0, 1, or -1

# Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    if not cap.isOpened():
        print("Error: Could not open video stream.")
    else:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            # Detect stuff and render
            ## Recolor image
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)

                # Visualize angle
                cv2.putText(image, str(angle), 
                    tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                print(angle)
            except:
                pass # maybe change further into project as we want to assess form accuracy

            # Reder detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )

            # print(results) to verify wireframe

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'): # Press 'Q' to exit
                break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1730036274.127859 11073052 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2
W0000 00:00:1730036274.325149 11208791 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730036274.348613 11208791 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


256.325869646843
256.67890839873917
257.3391105001881
99.44553114717924
7.06521463668371
49.89902864835929
11.551333293018793
256.11992654856937
82.2219389268033
65.6061969287295
56.93293119032914
49.82238372854248
45.41392987666378
47.82587246644122
44.65443432274204
39.04702722921635
29.910840305302585
27.904500768226498
28.387732564890836
33.64211215968946
38.946384963619415
44.94669447396984
71.4466150444185
66.7500325865353
29.09798423600518
29.84843054914785
242.83503666464884
85.0164207725136
76.00791499734554
69.39263571743432
46.337860767402894
38.76135400170905
37.10885824611203
50.64474342991777
256.1797073503074
256.83226186597244
257.00724688306764
62.66985072257261
93.78410602517404
209.20993476463335
3.6084322511000018
248.77849377906688
255.63183921688932
259.3875484809735
257.85253680808233
9.346848046069406
9.499669687567023
255.0627558322508
257.01622734468236
256.97098832232064
254.56254851355362
257.4436086874284
255.36376754158232
252.36794450661603
256.3638542236

# 4. Curl Counter #

In [44]:
# UPDATING code from above to count number of curls

# VIDEO FEED
cap = cv2.VideoCapture(1) # BEWARE might have to change number to 0, 1, or -1

#Curl Counter Variables
counter = 0
stage = None

# Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    if not cap.isOpened():
        print("Error: Could not open video stream.")
    else:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            # Detect stuff and render
            ## Recolor image
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)

                # Visualize angle
                cv2.putText(image, str(angle), 
                    tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # Curl counter logic
                if angle > 160:
                    stage = "Down"
                if angle < 30 and stage == "Down":
                    stage = "Up"
                    counter += 1
                    print(f"Curl count: {counter}")

                print(angle)
            except:
                pass # maybe change further into project as we want to assess form accuracy

            # Render curl counter into video feed
            ## Setup status box
            cv2.rectangle(image, (0,0),(225, 73), (245, 117, 16), -1) # blue box

            ## Rep data
            cv2.putText(image, 'REPS', (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            ## Stage data
            cv2.putText(image, 'STAGE', (65, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

            # Reder detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )

            # print(results) to verify wireframe

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'): # Press 'Q' to exit
                break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1730039101.039199 11073052 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2
W0000 00:00:1730039101.132481 11235156 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730039101.143761 11235156 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


251.47910678691022
249.9105835405225
245.67577506365845
235.3784098904298
250.57095797610936
253.99333988240744
247.5039316223992
248.53857922324934
250.52950489748588
249.27855430424123
Curl count: 1
0.1390235876974942
0.3369241686777801
3.9115668515996185
238.900458805776
237.4175692421654
224.78707327306003
Curl count: 2
2.4916804598515614
1.6018423126307224
10.11287592590991
10.439991665058805
7.2440548488574645
4.909994542244331
5.017807468075325
5.480111673363738
4.662306232828454
4.938039837892235
7.646532345545917
223.32436486451064
244.59858615878122
230.91806216642723
210.25200255823907
201.2775776527283
198.39661617665408
238.1807305561608
239.24500173759824
241.03342506540457
246.41398173230507
247.86706189474867
247.11703028213856
251.10748949600355
Curl count: 3
7.2955970987942935
17.366100800632076
6.5388979093465425
4.57393921329736
4.053595628331212
4.354609862011845
5.028881331231456
6.952595517433928
10.382550632442063
246.02341090370413
231.73985521243597
242.832970

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()